# LangChain Agent 스켈레톤 구현(Gemma)

In [ ]:
%%time
!pip install -q langchain langchain_community huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 6.3 MB/s eta 0:00:00
CPU times: user 1.53 s, sys: 309 ms, total: 1.84 s
Wall time: 8.22 s


### llama-cpp 설치

In [ ]:
#!pip uninstall -y llama-cpp-python
!pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu122
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 36.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.1 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.9-cp311-cp311-linux_x86_64.whl size=4123167 sha256=60ff29fe8f9ca999dd17974ecd0507973a24334c4a49eed5063824f2427cadfe
  Stored in directory: /root/.cache/pip/wheels/9e/8f/bf/148c8eb7d69021eccd6eae6444f3accd48347587054ffd24e5
Successfully built llama-cpp-python


In [ ]:
import os
from google.colab import userdata

# Colab Secrets에서 토큰 읽어오기
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
!huggingface-cli whoami

gshong
orgs:  LLM2506,aicmap


In [ ]:
import torch
# 주요 라이브러리 임포트
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_core.messages import SystemMessage

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

## 매뉴얼 파일 생성


In [ ]:
# 매뉴얼 파일 생성
manual_content = """
= DUV (Deep Ultraviolet) 공정 매뉴얼 =

문서 버전: 1.2
작성일: 2025-06-12

== 1. 개요 ==
DUV 리소그래피는 248nm 또는 193nm 파장의 빛을 사용하여 웨이퍼에 미세 회로 패턴을 형성하는 핵심 공정입니다.

== 2. 주요 단계 ==
2.1 웨이퍼 준비 (Wafer Preparation)
- 웨이퍼 세척 및 표면 처리

2.2 감광액 도포 (PR Coating)
- 스핀 코팅 방식을 사용하여 균일한 두께의 감광액(PR) 막을 형성합니다.

2.3 노광 (Exposure)
- 마스크에 설계된 패턴을 빛을 이용해 웨이퍼의 감광액 위로 전사시킵니다.
- **핵심 파라미터:** 노광 에너지(Dose), 초점(Focus)이 패턴의 정밀도를 결정합니다.
- 노광 장비의 정렬(Align) 정확도가 수율에 큰 영향을 미칩니다.

2.4 현상 (Development)
- 노광된 영역 또는 노광되지 않은 영역의 감광액을 선택적으로 제거하여 패턴을 완성합니다.
"""

with open("DUV_manual.txt", "w") as f:
    f.write(manual_content)

print("'DUV_manual.txt' 매뉴얼 파일 생성.")

def load_manual(path: str) -> str:
    """지정된 경로(path)의 텍스트 파일을 읽어 그 내용을 문자열로 반환합니다."""
    try:
        with open(path, 'r', encoding='utf-8') as f:
            return f.read()
    except FileNotFoundError:
        return f"오류: '{path}' 파일을 찾을 수 없습니다."

'DUV_manual.txt' 매뉴얼 파일 생성.


## 반도체 데이터베이스 정의


In [ ]:
# Tool 구현에 필요한 모든 데이터와 함수 정의

# 반도체 데이터베이스 정의: 'aliases' 키를 추가하여 영어 이름 포함
semiconductor_processes = {
    "포토리소그래피": {
        "aliases": ["photolithography", "photo"],
        "주요파라미터": "노광량(Exposure Dose), 초점(Focus), 감광액(PR) 두께",
        "설명": "반도체 웨이퍼 위에 회로 패턴을 형성하는 핵심 공정",
        "단위": "노광량(mJ/cm²), 초점(μm), PR두께(nm)",
        "중요도": "패턴 해상도와 정밀도를 결정하는 가장 중요한 공정"
    },
    "식각": {
        "aliases": ["etching", "etch"],
        "주요파라미터": "가스 유량, 압력(Pressure), RF 파워, 식각 시간",
        "설명": "불필요한 물질을 제거하여 원하는 패턴을 만드는 공정",
        "단위": "가스유량(sccm), 압력(mTorr), RF파워(W), 시간(min)",
        "중요도": "선폭 제어와 프로파일 형성에 핵심적"
    },
    "증착": {
        "aliases": ["deposition", "cvd", "pvd"],
        "주요파라미터": "온도, 압력, 전구체(Precursor) 유량",
        "설명": "웨이퍼 위에 얇은 막을 형성하는 공정",
        "단위": "온도(°C), 압력(Torr), 유량(sccm)",
        "중요도": "막질과 두께 균일성을 결정"
    }
}
# Tool로 사용할 함수들 정의

# 한국어 키와 영어 별칭(aliases)을 모두 검색
def process_lookup(query: str) -> str:
    """
    사용자 질문(query)에서 공정 이름(한/영)을 찾아, 해당 공정의 주요 파라미터, 설명, 단위, 중요도 등 상세 정보를 반환합니다.
    '포토리소그래피', '식각', '증착' 및 관련 영어 이름으로 정보를 조회할 수 있습니다.
    """
    print(f"🔍 [고급 검색] '{query}' 분석 중...")
    search_term = query.lower() # 검색어를 소문자로 변환하여 비교 정확도 높임

    for process_name, info in semiconductor_processes.items():
        # 한국어 공정 이름 또는 영어 별칭(aliases) 중에 일치하는 것이 있는지 확인
        if process_name in search_term or any(alias in search_term for alias in info.get("aliases", [])):
            detailed_info = f'''
====== {process_name} 공정 상세 정보 ======
- 주요 파라미터: {info['주요파라미터']}
- 측정 단위: {info['단위']}
- 공정 설명: {info['설명']}
- 중요도: {info['중요도']}
==========================================
'''
            print(f"[검색 성공] {process_name} 공정 정보 발견")
            return detailed_info

    available = ", ".join(semiconductor_processes.keys())
    result = f"해당 공정을 찾을 수 없습니다. 사용 가능한 공정: {available}"
    print(f"[검색 실패] {result}")
    return result

## Tool 리스트 정의


In [ ]:
# 2개의 Tool로 구성된 리스트 정의
tools = [
    Tool(
        name="Semiconductor Process DB Lookup",
        func=process_lookup,
        description="반도체 공정(포토리소그래피, 식각, 증착)에 대한 상세 정보(주요 파라미터, 설명, 단위, 중요도)를 찾을 때 사용합니다. 사용자 질문에 공정 이름이 포함되어 있어야 합니다."
    ),
    Tool(
        name="Manual Loader",
        func=load_manual,
        description="로컬 텍스트 파일 형식의 매뉴얼을 읽을 때 사용합니다. 'DUV_manual.txt'와 같이 정확한 파일 경로를 입력해야 합니다."
    )
]


## LLM 래퍼 설정  
> Wrapper는 LLM의 종류와 특성을 LangChain에 맞도록 숨겨주는 역할을 함, Interface 통일, LLM 교체를 쉽게 할 수 있음     
> OpenAI의 GPT-4 모델을 사용합니다. temperature=0으로 설정하여 일관성 있는 답변을 유도합니다.

In [ ]:
# LlamaCpp로 모델 로드 : GGUF 모델 다운로드
from huggingface_hub import hf_hub_download
from langchain_community.llms import LlamaCpp

model_repo = "unsloth/gemma-3-12b-it-qat-GGUF"
model_file = "gemma-3-12b-it-qat-Q4_0.gguf"

model_path = hf_hub_download(repo_id=model_repo, filename=model_file, local_dir="./")

llm = LlamaCpp(
    model_path=model_path,
    n_gpu_layers=40,  # GPU에 최대 레이어 오프로드 (8B 모델에 적합)
    n_batch=512,
    n_ctx=2048,  # 컨텍스트 길이
    f16_kv=True,  # FP16 메모리 최적화
    verbose=False, #True,
    temperature=0.7,
    max_tokens=256
)

gemma-3-12b-it-qat-Q4_0.gguf:   0%|          | 0.00/6.91G [00:00<?, ?B/s]

llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


In [ ]:
# 테스트
query = "반도체 공정에서 웨이퍼란 무엇인가요?"
response = llm.invoke(query)#, max_tokens=512)
print("Query:", query)
print("Response:", response)

Query: 반도체 공정에서 웨이퍼란 무엇인가요?
Response: 

웨이퍼는 반도체 칩을 만드는 데 사용되는 얇고 원형의 실리콘 조각입니다. 웨이퍼는 여러 개의 반도체 칩으로 나뉘며, 각 칩은 전자 장치에 사용됩니다. 웨이퍼는 다양한 크기로 제공되며 가장 일반적인 크기는 300mm(12인치)입니다.

다음은 웨이퍼의 몇 가지 주요 특징입니다.

* **재료:** 웨이퍼는 일반적으로 실리콘으로 만들어집니다. 실리콘은 반도체 특성을 가지고 있어 전자 장치의 핵심 재료로 사용됩니다.
* **크기:** 웨이퍼의 크기는 직경을 나타냅니다. 가장 일반적인 크기는 300mm(12인치)이지만, 더 작은 크기의 웨이퍼(예: 200mm, 150mm)도 사용됩니다.
* **표면 품질:** 웨이퍼 표면은 매우 평평하고 깨끗해야 합니다. 불량은 칩 생산에 문제를 일으킬 수 있습니다.
* **정렬:** 웨이퍼는 정렬되어야 하며, 이는 다양한 공정을 통해 칩


## LangChain Agent 생성
> agent: 사용할 에이전트의 유형을 지정합니다. (e.g., ZERO_SHOT_REACT_DESCRIPTION)   
 tools: 에이전트가 사용할 도구 목록  
 llm: 에이전트의 추론을 담당할 언어 모델  
 verbose=True: 에이전트의 생각(Thought)과 행동(Action) 과정을 모두 출력하여 디버깅에 용이하게 합니다.

In [ ]:
# 시스템 프롬프트 내용 정의 : 역할과 답변형식 지시
system_message_content = "You are a helpful assistant specializing in semiconductor processes. Your primary goal is to provide accurate and detailed information to the user. All final answers must be written clearly and politely in Korean."
agent_kwargs = {"system_message": SystemMessage(content=system_message_content)}

# LangChain Agent 생성
# 채팅 모델에 더 적합한 'CHAT_ZERO_SHOT_REACT_DESCRIPTION' 에이전트 사용 및 kwargs 전달
agent_final_korean = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, # 채팅 모델에 최적화된 ReAct 에이전트
    verbose=True,
    handle_parsing_errors=True,
    agent_kwargs=agent_kwargs ) # 시스템 프롬프트 전달

## 검증 쿼리 실행  

In [ ]:
# 검증 쿼리 실행
agent_final_korean.verbose = True  # 중간 과정 설정
print("--- 검증 :  단일 Tool 테스트 ---")
response = agent_final_korean.run("식각 공정의 상세 정보와 측정 단위를 알려줘.")
print("\n최종 답변 :")
print(response)

--- 검증 :  단일 Tool 테스트 ---


> Entering new AgentExecutor chain...
Thought: I need to find the details of etching process and its units.
Action:
```json
{
  "action": "Semiconductor Process DB Lookup",
  "action_input": "etching"
}
```
🔍 [고급 검색] 'etching' 분석 중...
[검색 성공] 식각 공정 정보 발견

Observation: 
====== 식각 공정 상세 정보 ======
- 주요 파라미터: 가스 유량, 압력(Pressure), RF 파워, 식각 시간
- 측정 단위: 가스유량(sccm), 압력(mTorr), RF파워(W), 시간(min)
- 공정 설명: 불필요한 물질을 제거하여 원하는 패턴을 만드는 공정
- 중요도: 선폭 제어와 프로파일 형성에 핵심적

Thought: I now have the detailed information and units for etching process.
Final Answer: 주요 파라미터는 가스 유량(sccm), 압력(mTorr), RF 파워(W), 식각 시간(min)입니다.


> Finished chain.

최종 답변 :
주요 파라미터는 가스 유량(sccm), 압력(mTorr), RF 파워(W), 식각 시간(min)입니다.


In [ ]:
# 검증 쿼리 실행
agent_final_korean.verbose = False  # 중간 과정 설정
print("--- 검증 :  단일 Tool 테스트 ---")
response = agent_final_korean.run("식각 공정의 상세 정보와 측정 단위를 알려줘.")
print("\n최종 답변 :")
print(response)

--- 검증 :  단일 Tool 테스트 ---
🔍 [고급 검색] '식각' 분석 중...
[검색 성공] 식각 공정 정보 발견

최종 답변 :
가스 유량(sccm), 압력(mTorr), RF 파워(W), 시간(min)을 사용한다. 불필요한 물질을 제거하여 원하는 패턴을 만드는 공정이다.


In [ ]:
agent_final_korean.verbose = True   # 중간 과정 설정
print("--- 검증 :  단일 Tool 테스트 ---")
response = agent_final_korean.run("포토리소그래피 단계의 주요 파라미터를 알려줘.")
print("\n최종 답변 :")
print(response)

--- 검증 :  단일 Tool 테스트 ---


> Entering new AgentExecutor chain...
Thought: I need to find the parameters of photolithography. I will use Semiconductor Process DB Lookup for this task.
Action:
```json
{
  "action": "Semiconductor Process DB Lookup",
  "action_input": "Photolithography"
}
```
🔍 [고급 검색] 'Photolithography' 분석 중...
[검색 성공] 포토리소그래피 공정 정보 발견

Observation: 
====== 포토리소그래피 공정 상세 정보 ======
- 주요 파라미터: 노광량(Exposure Dose), 초점(Focus), 감광액(PR) 두께
- 측정 단위: 노광량(mJ/cm²), 초점(μm), PR두께(nm)
- 공정 설명: 반도체 웨이퍼 위에 회로 패턴을 형성하는 핵심 공정
- 중요도: 패턴 해상도와 정밀도를 결정하는 가장 중요한 공정

Thought: I now know the parameters of photolithography.
Final Answer: 주요 파라미터는 노광량, 초점, 감광액 두께입니다. 측정 단위는 각각 mJ/cm², μm, nm 입니다.


> Finished chain.

최종 답변 :
주요 파라미터는 노광량, 초점, 감광액 두께입니다. 측정 단위는 각각 mJ/cm², μm, nm 입니다.


In [ ]:
# 검증 쿼리 실행 : 복합 질문 테스트

agent_final_korean.verbose = False  # 중간 과정 설정
print("\n\n--- 검증 : 복합 질문(Multi-Tool) 테스트 ---")
query_complex = "DUV 공정 매뉴얼에서 노광 단계 설명을 요약하고, 포토리소그래피 공정의 중요도도 알려줘."
response = agent_final_korean.run(query_complex)
print("\n최종 답변:")
print(response)



--- 검증 : 복합 질문(Multi-Tool) 테스트 ---
🔍 [고급 검색] '포토리소그래피' 분석 중...
[검색 성공] 포토리소그래피 공정 정보 발견

최종 답변:
DUV 리소그래피는 248nm 또는 193nm 파장의 빛을 사용하여 웨이퍼에 미세 회로 패턴을 형성하는 핵심 공정입니다. 노광 단계에서는 마스크에 설계된 패턴을 빛을 이용해 웨이퍼의 감광액 위로 전사시킵니다. 노출 에너지


In [ ]:
agent_final_korean.verbose = True
response = agent_final_korean.run("증착 공정의 중요도는?")
print("\n최종 답변:")
print(response)



> Entering new AgentExecutor chain...
Thought: 증착 공정의 중요도를 알아보기 위해 Semiconductor Process DB Lookup 도구를 사용합니다.
Action:
```json
{
  "action": "Semiconductor Process DB Lookup",
  "action_input": "증착"
}
```
🔍 [고급 검색] '증착' 분석 중...
[검색 성공] 증착 공정 정보 발견

Observation: 
====== 증착 공정 상세 정보 ======
- 주요 파라미터: 온도, 압력, 전구체(Precursor) 유량
- 측정 단위: 온도(°C), 압력(Torr), 유량(sccm)
- 공정 설명: 웨이퍼 위에 얇은 막을 형성하는 공정
- 중요도: 막질과 두께 균일성을 결정

Thought: 증착 공정의 중요도를 알았으니 최종 답변을 합니다.
Final Answer: 막질과 두께 균일성을 결정합니다.

> Finished chain.

최종 답변:
막질과 두께 균일성을 결정합니다.


# HF Transformer Tool-Calling Agent 구현

In [ ]:
import os
import json
from google.colab import userdata
from langchain.agents import AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool # Tool 생성을 위한 데코레이터

## Tool로 사용할 함수 정의 (@tool 데코레이터 사용)


In [ ]:
# Tool로 사용할 함수 정의 (@tool 데코레이터 사용)
# (@tool) 데코레이터를 사용하면 함수 자체가 LangChain Tool 객체로 변환됨
# 함수의 docstring이 자동으로 tool의 description이 되어 매우 편리합니다.
@tool
def process_lookup(query: str) -> str:
    """사용자 질문에서 '포토리소그래피', '식각', '증착' 같은 공정 이름을 찾아, 해당 공정의 상세 정보(파라미터, 설명, 단위, 중요도)를 반환합니다."""
    for process_name, info in semiconductor_processes.items():
        if process_name in query:
            return json.dumps(info, ensure_ascii=False) # 결과를 JSON 문자열로 반환하여 모델이 파싱하기 쉽게 함
    return "해당 공정을 찾을 수 없습니다."

@tool
def load_manual(path: str) -> str:
    """'DUV_manual.txt'와 같이 로컬 파일 시스템에 있는 텍스트 형식의 매뉴얼을 읽어 그 내용을 문자열로 반환합니다."""
    try:
        with open(path, 'r', encoding='utf-8') as f:
            return f.read()
    except FileNotFoundError:
        return f"오류: '{path}' 파일을 찾을 수 없습니다."

# 도구 리스트 정의
tools = [process_lookup, load_manual]

## LLM Wrapper , Agent 생성

In [ ]:
# llm = LlamaCpp(
#     model_path=model_path,
#     n_gpu_layers=40,  # GPU에 최대 레이어 오프로드 (8B 모델에 적합)
#     n_batch=512,
#     n_ctx=2048,  # 컨텍스트 길이
#     f16_kv=True,  # FP16 메모리 최적화
#     verbose=False, #True,
#     temperature=0.7,
#     max_tokens=256
# )

In [ ]:
# 프롬프트 템플릿 설정
# tool-calling을 잘 수행하도록 역할과 지시사항을 상세하게 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 반도체 공정 전문가 AI 어시스턴트다. 사용자의 질문을 해결하기 위해 주어진 도구를 적극적으로 사용해야 한다. 모든 최종 답변은 반드시 상세하고 친절한 한국어로 작성해야 한다."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"), # 에이전트의 중간 작업 내역(생각, 도구 호출 등)을 위한 자리
])

In [ ]:
# LangChain Agent 생성
# 채팅 모델에 더 적합한 'CHAT_ZERO_SHOT_REACT_DESCRIPTION' 에이전트 사용 및 kwargs 전달
agent_tool_calling = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, # 채팅 모델에 최적화된 ReAct 에이전트
    verbose=True,
    handle_parsing_errors=True,
    agent_kwargs=agent_kwargs ) # 시스템 프롬프트 전달

# 에이전트 실행 및 검증


In [ ]:
# 에이전트 실행 및 검증
print("\n\n--- 복합 질문(Multi-Tool) 테스트 ---")
complex_query = "DUV 공정 매뉴얼에서 노광 단계 설명을 요약하고, 포토리소그래피 공정의 중요도도 알려줘."

# .invoke()를 사용하여 에이전트 실행
agent_tool_calling.verbose = False  #
response = agent_tool_calling.invoke({"input": complex_query})

print("\n\n최종 답변:")
print(response['output'])